# Summary

This notebook will show the following operations for a `Tile` object:
 
 - Creating a tile object from a shape file.
 - Loading data into the tile object for both world clim and cru-jra data.
 - Saving the tile object to a folder.

When a tile is created, it is empty, but has an extent as determined from the 
shape file used to create the tile. 

When loading data, it is assumed that the world clim and cru data sources have
extents fully encompasing the tile. When the data is loaded it is clipped to the
extents of the tile, optionally including a buffer beyond the tile extents
(reccomened; helps with warping and reprojection).

When a tile is saved, all the datasets that the tile holds are saved to disk in 
an appropriately dimensioned netcdf file. Additionally there is a manifest file
which will have metadata (tile index, extents, resolution, crs, etc).

In [ ]:
# For development...
# %load_ext autoreload
# %autoreload 2

from pathlib import Path
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from temds.datasources import worldclim
from temds.datasources import crujra

from temds import tile

# Creating a `Tile` object

Here we setup a few parameters defining the tile we want to create. This includes
the:
  - Tile "index" (horizontal and vertical offsets from lower left corner). 
  The shape file should be annotated with these horizontal and vertical indices 
  at attributes for each tile.
  - Year range to work with (typically all the data you have, but for 
    development and testing you can make it shorter).

Then we read the bounds for the tile from the shape file.

In [ ]:
start_year = 1901
end_year = 2023
c_tile = (0, 8)
tile_index = gpd.read_file('working/tile_index_annotated.shp')
hdx = tile_index['H'] == c_tile[0]
vdx = tile_index['V'] == c_tile[1]
bounds = tile_index[vdx & hdx].bounds
minx, maxx, miny, maxy = bounds[['minx','maxx','miny','maxy']].iloc[0]
minx, maxx, miny, maxy

Next we create a tile object. At this point the tile object is "empty" - there is no data loaded yet.

In [ ]:
mytile = tile.Tile(c_tile, bounds, 4000, tile_index.crs, buffer_px=20)
mytile.verbose = True

In [ ]:
mytile.buffer_pixels

In [ ]:
mytile.extent

# Loading data to the `Tile` object

Once we have created a `Tile` object we can load some data to it. 

Before we can import (load) the data into the `Tile` object, we need to open and create the datasets. For this tile, we will be using WorldClim and CRU-JRA data.

## WorldClim

The WorldClim data is relatively straightforward - it is a single NetCDF file.

In [ ]:
wc_arctic = worldclim.WorldClim('working/02-arctic/worldclim/worldclim-arctic.nc')
print(f"The CRS for the WorldClim dataset is: {wc_arctic.dataset.rio.crs}")

mytile.import_normalized('worldclim', wc_arctic, flip_y = True)

mytile.data['worldclim']
#del(wc_arctic)

In [ ]:
print(f"WorldClim dimensions: {mytile.data['worldclim'].dims}")
print(f"Resolution: {mytile.resolution}")
tile_x_size = (mytile.extent['maxx']-mytile.extent['minx'])/mytile.resolution
tile_y_size = (mytile.extent['maxy']-mytile.extent['miny'])/mytile.resolution
print(f"Tile Size: {tile_x_size.values} x {tile_y_size.values} pixels")
print(f"Tile Buffer Pixels: {mytile.buffer_pixels}")


Now we can plot and look at the data to double check.

In [ ]:
fig, axes= plt.subplots (1,1, dpi=100)

im = axes.imshow(mytile.data['worldclim']['tmax'].data[0], origin='lower')
fig.colorbar(im, ax=axes)
axes.set_title('wc example')

## CRU-JRA
The CRU-JRA data is a little more complicated as it is a series of NetCDF files.

First we can do some poking around to see what files we have available:

In [ ]:
i = sorted(list(Path('working/02-arctic/cru-jra-fixed/').glob('*.nc')))
print(f"Found {len(i)} files.")
print(i[0])
print("...")
print(i[-1])

It is up to the user to make sure they have enough files. You must have a continuous range covering your `start_year` through your `end_year` that you set above.

This will create a list of `crujra.AnnualDaily` objects. The data is resampled from 6 hour to daily when these objects are created.

> (~4 minutes and 18GB RAM on 8 core machine with 32GB RAM and SSD)

### Resample from 6h to daily

In [ ]:
# Opens cru files, reads into memory, then we have a list of cru objects at 
# the end...
annual_list = []
file_list = sorted(list(Path('working/02-arctic/cru-jra-25/').glob('*.nc')))
print(f"culling by year: {start_year} - {end_year}")
for cru_file in file_list:
    year = int(cru_file.name.split('.')[-4])
    if year >= start_year and year <= end_year:
        temp = crujra.AnnualDaily(year, cru_file, verbose=False, force_aoi_to='tmax', aoi_nodata=np.nan)
        # temp.reproject(tile_index.crs.to_wkt())
        annual_list.append(temp)
    else:
        continue
print(len(annual_list))

Now we have a list of `crujra.AnnualDaily` objects, and we can turn that list into a `crujra.AnnualTimeSeries` object. This `AnnualTimeSeries` object has some special features like being able to index by year.

In [ ]:
cru_arctic_ts = crujra.AnnualTimeSeries(annual_list)

We can plot this data to confirm that it is the expected shape. The data should be monthly and have all variables present. The data should cover the entire acrtic and is un-projected, so it doesn't look great, but you get the idea...

In [ ]:
print("The CRS is: ", cru_arctic_ts[1901].dataset.rio.crs)
print("Time dimension shape: ", cru_arctic_ts[1901].dataset.time.shape)
print("The data variables present: \n", cru_arctic_ts[1901].dataset.data_vars)


In [ ]:
fix, axes = plt.subplots(2,1)
tmax_im = axes[0].imshow(cru_arctic_ts[start_year].dataset['tmax'][0])
pre_im = axes[1].imshow(cru_arctic_ts[start_year].dataset['pre'][0], cmap='Greens')
c1 = plt.colorbar(tmax_im, ax=axes[0])
c2 = plt.colorbar(pre_im, ax=axes[1])

### Clip all files down to tile extents

Now that we have this data loaded (and resampled), we can clip out the data for our tile. For this we use the "`Tile.import_normalized(...)` function. 

> This is **SLOW** (40 minutes on 8 core 32GB RAM laptop; I think it fills the RAM and then slows way down using the swap memory...)

In [ ]:
### serial
mytile.verbose = True
mytile.import_normalized('cru_AnnualTimeSeries', cru_arctic_ts)

### Parallel

# from dask.distributed import Client, LocalCluster
# import joblib

# # replace with whichever cluster class you're using
# # https://docs.dask.org/en/stable/deploying.html#distributed-computing
# cluster = LocalCluster(n_workers=24) # <<< change if needed
# # connect client to your cluster
# client = Client(cluster)
# print(client.dashboard_link)

# start = datetime.now()
# with joblib.parallel_config(backend="dask", n_jobs=20, verbose=1):
#     mytile.import_normalized('cru_AnnualTimeSeries', cru_arctic_ts, parallel=True)


### end parallel

#del(cru_arctic_ts)

Great! Now we have a tile with all the data loaded. Check it out:

In [ ]:
print(mytile.data.keys())
print(mytile.data['worldclim'].dims)

cru_first_year = mytile.data['cru_AnnualTimeSeries'][start_year]
cru_last_year = mytile.data['cru_AnnualTimeSeries'][end_year]
print(cru_first_year.dataset.dims)
print(cru_last_year.dataset.dims)

print(cru_first_year.dataset.rio.crs)
print(mytile.data['worldclim'].rio.crs )


# Saving the `Tile`

The directory is created based on the tile's index (horizontal and vertical, i.e.: H00_V08)

> **SLOW**, maybe 50 minutes

In [ ]:
mytile.toggle_verbose()
mytile.save('working/03-tiles', overwrite=True, clear_existing=False)


# Extra Stuff (plots, etc)

In [ ]:
print(mytile)

In [ ]:
import pandas as pd
print(f"{wc['x'].min().values}")
print(f"{wc['x'].max().values}")
print(f"{wc['y'].min().values}")
print(f"{wc['y'].max().values}")
print(bounds)
wc_bnds = pd.DataFrame({'minx': [wc['x'].min().values], 'maxx': [wc['x'].max().values],
                        'miny': [wc['y'].min().values], 'maxy': [wc['y'].max().values]})
print(wc_bnds)
print(wc_bnds['minx'][0] - bounds['minx'].values[0])
print(wc_bnds['maxx'][0] - bounds['maxx'].values[0])
print(wc_bnds['miny'][0] - bounds['miny'].values[0])
print(wc_bnds['maxy'][0] - bounds['maxy'].values[0])

print(wc['x'][0]-wc['x'][1]) # 4000


In [ ]:
plt.close()
fig, axes = plt.subplots(1,2)
axes[0].plot(mytile.data['worldclim']['x'])
axes[0].plot(mytile.data['cru_AnnualTimeSeries'][start_year].dataset['x'])
axes[1].plot(mytile.data['worldclim']['y'])
axes[1].plot(mytile.data['cru_AnnualTimeSeries'][start_year].dataset['y'])